現在のプログラミング状況　2025/06/20


*   リスト項目
*   リスト項目



RAGデータストアの自動構築

In [1]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()


In [4]:
!pip install serpapi
!pip install faiss-cpu
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [9]:
#RAGデータストアの自動構築
import requests
from bs4 import BeautifulSoup
import openai
from serpapi.google_search import GoogleSearch
import faiss
import numpy as np


# 環境変数にAPIキーを設定（直接代入でも可）
SERPAPI_KEY = "292b40a122ffbca94d7ec12ce87b97bf5d723e4f9053c9ad3b506f2595606fa8"
#OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"
#openai.api_key = OPENAI_API_KEY

def web_search(query, num_results=3):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_KEY,
        "num": num_results,
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return [r["link"] for r in results.get("organic_results", [])[:num_results]]

def fetch_and_clean(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")
        texts = [p.get_text() for p in soup.find_all("p")]
        return "\n".join(texts).strip()
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return ""

def summarize_text(text, max_tokens=300):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "以下の長文をRAG用に簡潔に要約してください。"},
            {"role": "user", "content": text[:3000]},  # OpenAI token制限を考慮
        ],
        max_tokens=max_tokens,
        temperature=0.3
    )
    return response["choices"][0]["message"]["content"]

def get_embedding(text):
    response = openai.Embedding.create(
        input=[text],
        model="text-embedding-3-small"
    )
    return np.array(response["data"][0]["embedding"], dtype="float32")

def create_vector_store(docs):
    dim = len(docs[0][1])
    index = faiss.IndexFlatL2(dim)
    texts = []
    for text, vector in docs:
        index.add(np.array([vector]))
        texts.append(text)
    return index, texts

def rag_respond(query, index, texts):
    q_vector = get_embedding(query)
    D, I = index.search(np.array([q_vector]), k=1)
    context = texts[I[0][0]]

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "以下のコンテキストを元にユーザーの質問に答えてください。"},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"},
        ],
        temperature=0.5,
        max_tokens=300
    )
    return response["choices"][0]["message"]["content"]

# === 実行例 ===
if __name__ == "__main__":
    query = "最新のChatGPTのバージョンは？"
    urls = web_search(query)
    print("検索URL:", urls)

    docs = []
    for url in urls:
        text = fetch_and_clean(url)
        if text:
            summary = summarize_text(text)
            embedding = get_embedding(summary)
            docs.append((summary, embedding))

    if docs:
        index, texts = create_vector_store(docs)
        answer = rag_respond(query, index, texts)
        print("\n🧠 回答:\n", answer)
    else:
        print("❌ 有効な情報が取得できませんでした。")

❌ 有効な情報が取得できませんでした。
